In [ ]:
!pip install -U -q "google-genai==1.7.0"
from google import genai
from google.genai import types
from IPython.display import HTML, Markdown, display
from google.api_core import retry
is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})
genai.models.Models.generate_content = retry.Retry(
    predicate=is_retriable)(genai.models.Models.generate_content)

In [ ]:
#Gets patient information
patient_file=input("Please input the patient information here: ")

#Gets API Key
kaggle_sec_bool=input("Is your Google API Key in a Kaggle Secret (Y or N): ").upper()
if kaggle_sec_bool=="N":
    api_key = input("Please input your Google API Key: ")
elif kaggle_sec_bool=="Y":
    sec_name=input("What is the name of your API Key Secret (use the EXACT case): ")
from kaggle_secrets import UserSecretsClient
    api_key = UserSecretsClient().get_secret(sec_name)

#Sets up client
client=genai.Client(api_key=api_key)

#Few Shot Prompting
#Structured output/JSON mode/controlled generation

#I am providing different examples for the AI to base its response off of.
few_shot_prompt = """Parse a hospital patient's file into a valid JSON.

Example:
A 5 year old male patient has a common cold and has symptoms including a runny nose and sneezing. He has a temprature of 98 degrees Farenheit and an SpO2 of 98. He weighs 40 pounds.

JSON Response:


```
{
"age" : 5,
"gender": "male",
"disease": "common cold",
"symptoms": ["sneezing", "runny nose"],
"temprature": "98 (F)",
"SpO2": "98",
"weight" : "40 lbs"
}
```

Example: A 13 year old has a stomach bug, is vomiting, and has had extreme loss of appetite. They are running a temprature of 37 degrees Celsius. They are allergic to peanuts and are not on any current medications. They weigh 90 kilograms.

```
{
"age" : 13,
"disease": "stomach bug",
"symptoms": ["vomiting", "appetite loss"],
"temprature": "37 (C)",
"allergies" : ["peanuts"],
"current medications" : null,
"weight" : "90 kgs"
}
```

Example: "The patient is complaining of sudden onset of sharp chest pain radiating to the left arm, accompanied by sweating and dizziness. They report no known allergies, are 55 years old, and identify as male."
```
{
"age" : 55,
"disease": null,
"symptoms": ["radiating chest pain (left arm)","sweating","dizziness"],
"gender": "male"
"allergies" : null,
}
```
"""


data=client.models.generate_content(
    model='gemini-2.0-flash',
    config=types.GenerateContentConfig(
        temperature=0.0,
        top_p=1.0),
    contents=[few_shot_prompt,patient_file]
).text

print(data)

#Groundings with Search

#This config helps us enable Google Search
config_with_search = types.GenerateContentConfig(
    tools=[types.Tool(google_search=types.GoogleSearch())],
)
recs=Markdown(client.models.generate_content(
    model='gemini-2.0-flash',
    config=config_with_search,
    contents=[data, patient_file, "You are a medical professional. Advise the patient and/or their legal guardians what they can do in order to help the patient? Remember, you are the medical professional in this scenario. Suggest things for professionals to do and other changes that the patient's side can do, such as lifestyle. Also try and adress the symptoms. Assume they have read the JSON data. If the official diagnosis is not given (or disease is marked as null), generate personalized plans for them and come up with potential diagnoses. State the specific disease, not just a broad category. Put your diagnoses in order of whatever you think is most likely, your first one should be the most likely."]
    ).text)
display(recs)

#Creates chat
chat = client.chats.create(model='gemini-2.0-flash',config=config_with_search)
query=input("\nWhat is your message to DoctorBot? Type EXIT to exit: ")
keep_running=True
if query.upper()=="EXIT":
    print("Ok, nice talking to you!")
    keep_running=False
else:
    response = chat.send_message(f"This is the file - {patient_file}, this is JSON data - {data}, these are the recommendations - {recs}. This is the query - {query}.").text
    display(Markdown(response))
while keep_running:
    print("\n")
    query=input("\nWhat is your message to DoctorBot? Type EXIT to exit: ")
    if query.upper()=="EXIT":
        print("Ok, nice talking to you!")
        break
    response=chat.send_message(query).text
    display(Markdown(response))